# goal
\
As part of the [**D**ata **S**yncing **G**izmo](https://github.com/HRDAG/dsg) development, we'll be using [Pydantic](https://docs.pydantic.dev/2.10/) objects to standardize the objects and associated testing.

I'm new to `pydantic`, so I'm following a [guide](https://www.geeksforgeeks.org/introduction-to-python-pydantic-library/) that has some organized examples I read through and borrowed the initial object structure from before diving further into the [docs](https://docs.pydantic.dev/2.10/).

The first two applications that come to mind and I'd like to explore are:
1. Excel file model (for initial processing of input data)
2. Entity model (for light standardization / de-duplication / entity resolution)

# setup

In [1]:
# dependencies
from pathlib import PosixPath
import pandas as pd
from pydantic import BaseModel, field_validator

In [2]:
# support methods
def cleanname(colname):
    form = colname.lower()
    swap = '.: '
    for char in swap: form = form.replace(char, '_')
    return form


class xlsx(BaseModel):
    filename: PosixPath | str
    sheets: None | list
    sheet: None | str

    @field_validator('filename')
    def checkpath(cls, value):
        if not PosixPath(value).exists():
            raise ValueError(f'Cannot find file with path {value}')
        return value

    def setsheets(self):
        known = pd.ExcelFile(self.filename).sheet_names
        if not len(known) >= 1:
            raise ValueError(f'Expected {self.filename} to have at least one read-able sheet of data.')
        if not self.sheets: self.sheets = known
        if sorted(known) != sorted(self.sheets):
            raise ValueError(f'Expected {self.filename} to have sheets {self.sheets} but found {known}.')
        return self

    def setsheet(self):
        if not self.sheet: self.sheet = self.sheets[0]
        if self.sheet not in self.sheets: raise ValueError(f'\
            {self.sheet} not found in {self.filename} with sheets {self.sheets}')
        return self

    def toframe(self):
        self.setsheets()
        self.setsheet()
        df = pd.read_excel(io=self.filename, sheet_name=self.sheet)
        df.rename(columns={c: cleanname(colname=c) for c in df.columns}, inplace=True)
        return df


def loadxl(xlfile, sheets=None, sheet=None):
    file = xlsx(filename=xlfile, sheets=sheets, sheet=sheet)
    df = file.toframe()
    return df

In [3]:
# main
xlf = "/Users/home/Downloads/211 and 212.5 cases with strike priors.xlsx"

# testing stuff we expect to pass

## real file, no other args

In [4]:
justfile = loadxl(xlfile=xlf)

## real file and sheet

In [5]:
wsheet = loadxl(xlfile=xlf, sheet='Sheet1')

## real file, no args == real file, real sheet

In [6]:
assert justfile.equals(wsheet)

# testing stuff we expect to fail

## fake file

In [7]:
loadxl(xlfile="~/Downloads/fake.xlsx")

ValidationError: 1 validation error for xlsx
filename
  Value error, Cannot find file with path ~/Downloads/fake.xlsx [type=value_error, input_value='~/Downloads/fake.xlsx', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

## real file but fake sheet

In [8]:
loadxl(xlfile=xlf, sheet='FakeSheetName')

ValueError:             FakeSheetName not found in /Users/home/Downloads/211 and 212.5 cases with strike priors.xlsx with sheets ['Sheet1']